# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Importing all necessary Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
   

/home/workspace


#### Processing the files to create the data file csv `event_datafile_new.csv` that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### checking the numers of rows in the file `event_datafile_new.csv`

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a  keyspace called `sparkify`

In [7]:
# creating the keyspace

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exeption as error:
    print(error)

#### setting the respective keyspace for `sparkify`

In [8]:
# setting the keyspace

try:
    session.set_keyspace('sparkify')
except Exception as error:
    print(error)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Creating a database table called `songs` plus data definition

In [9]:
# creating the database including all attributes plus definition

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songs (
            session_id INT,
            item_in_session INT,
            artist_name TEXT,
            song_title TEXT,
            song_length FLOAT,
            PRIMARY KEY (session_id, item_in_session)
        )
    """)
except Exception as error:
    print(error)                    

#### open file `event_datafile_new.csv` and insert data into `songs`

In [10]:
# opening the csv-file and doing the insert

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# executing a verification query

query = """
    SELECT artist_name, song_title, song_length FROM songs
    WHERE session_id = 338 AND item_in_session = 4
"""
try:
    songs_results = session.execute(query)
except Exception as error:
    print(error)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### create another database table called `users` plus doing the data definition

In [12]:
# creating the database including all attributes plus definition

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist_name TEXT,
            song_title TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        )
    """)
except Exception as error:
    print(error)                    

#### open file `event_datafile_new.csv` and insert data into `users`

In [13]:
# opening the csv-file and doing the insert

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    #scipping the header
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

#### proceed a query to verifiy the inserted data into table `users`

In [14]:
# executing a verification query

query = """
    SELECT artist_name, song_title, first_name, last_name FROM users
    WHERE user_id = 10 AND session_id = 182
"""
try:
    users_results = session.execute(query)
except Exception as error:
    print(error)

#### verification function

In [15]:
df = pd.DataFrame(users_results._current_rows)
print(df)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


#### Creating a database table called `users_per_song` plus data definition

In [16]:
# creating the database including all attributes plus definition

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS users_per_song (
            song_title TEXT,
            user_id INT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY (song_title, user_id)
        )
    """)
except Exception as error:
    print(error)  

#### open file `event_datafile_new.csv` and insert data into `users_per_song`

In [17]:
# opening the csv-file and doing the insert

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_per_song (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### proceed a query to verifiy the inserted data into table `users_per_song`

In [18]:
# executing a verification query

query = """
    SELECT first_name, last_name FROM users_per_song
    WHERE song_title = 'All Hands Against His Own'
"""
try:
    results = session.execute(query)
except Exception as error:
    print(error)

#### verification function

In [19]:
df = pd.DataFrame(results._current_rows)
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [20]:
# droping all database tables which were created during  the session

try:
    session.execute("DROP TABLE IF EXISTS songs")
    session.execute("DROP TABLE IF EXISTS users")
    session.execute("DROP TABLE IF EXISTS users_per_song")
except Exception as error:
    print(error)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()